In [ ]:
#| include: false
from nbdev.showdoc import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| include: false
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.prune.all import *
from fasterai.core.criteria import *
import torch_pruning as tp
from torch_pruning.pruner import function
import torch_pruning as tp

import torch
import torch.nn as nn
import torch.nn.functional as F

## Overview

**Structured Pruning** removes entire filters, channels, or layers from neural networks, resulting in genuinely smaller and faster models. Unlike sparsification (which zeros individual weights), pruning physically removes parameters.

### Why Use Structured Pruning?

| Approach | What's Removed | Model Size | Speed Benefit | Hardware |
|----------|----------------|------------|---------------|----------|
| Sparsification | Individual weights | Same | Requires sparse support | Specialized |
| **Structured Pruning** | Entire filters | **Smaller** | **Immediate** | Standard |

### Key Benefits

- **Real speedup** - Fewer parameters = faster inference on any hardware
- **Smaller models** - Reduced memory footprint for deployment
- **Gradual pruning** - Remove filters progressively during training
- **Flexible targeting** - Global or local pruning strategies

## 1. Setup and Baseline

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

First, train a baseline ResNet-18 to establish expected performance:

In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)
learn.unfreeze()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.654544,0.395677,0.853857,00:02


In [ ]:
base_macs, base_params = tp.utils.count_ops_and_params(learn.model, torch.randn(1,3,224,224).to(default_device()))

## 2. Training with PruneCallback

Now let's train with gradual filter pruning. We'll remove 40% of filters using a one-cycle schedule:

Configuration:
- **`pruning_ratio=40`** - Remove 40% of filters
- **`context='global'`** - Remove least important filters from anywhere in the network
- **`criteria=large_final`** - Keep filters with largest final weights
- **`schedule=one_cycle`** - Gradually increase pruning following one-cycle pattern

In [ ]:
pruner = Pruner(
learn.model,
criteria=large_final,
pruning_ratio=40, 
context='global',
iterative_steps=3, 
schedule=one_cycle._scheduler,
)

Ignoring output layer: Linear(in_features=512, out_features=2, bias=False)
Total ignored layers: 1


In [ ]:
pr_cb = PruneCallback(pruning_ratio=40, context='global', criteria=large_final, schedule=one_cycle)
learn.fit_one_cycle(10, cbs=pr_cb)

Ignoring output layer: Linear(in_features=512, out_features=2, bias=False)
Total ignored layers: 1


epoch,train_loss,valid_loss,accuracy,time
0,0.354357,0.313860,0.882950,00:02
1,0.289704,0.273885,0.891746,00:02
2,0.242167,0.244028,0.901218,00:03
3,0.219180,0.425877,0.770636,00:03
4,0.153443,0.236760,0.903248,00:03
5,0.175014,0.257431,0.900541,00:02
6,0.169409,0.232646,0.912720,00:03
7,0.147067,0.240273,0.910014,00:02
8,0.136972,0.226695,0.916779,00:02
9,0.125311,0.222924,0.916779,00:02


Sparsity at the end of epoch 0: 0.39%
Sparsity at the end of epoch 1: 1.54%
Sparsity at the end of epoch 2: 5.60%
Sparsity at the end of epoch 3: 15.91%
Sparsity at the end of epoch 4: 29.13%
Sparsity at the end of epoch 5: 36.64%
Sparsity at the end of epoch 6: 39.12%
Sparsity at the end of epoch 7: 39.79%
Sparsity at the end of epoch 8: 39.96%
Sparsity at the end of epoch 9: 40.00%


In [ ]:
pruned_macs, pruned_params = tp.utils.count_ops_and_params(learn.model, torch.randn(1,3,224,224).to(default_device()))

## 3. Measuring Compression

The pruned model has fewer parameters and requires less compute:

In [ ]:
print(f'The pruned model has {pruned_macs/base_macs:.2f} the compute of original model')

The pruned model has 0.63 the compute of original model


In [ ]:
print(f'The pruned model has {pruned_params/base_params:.2f} the parameters of original model')

The pruned model has 0.18 the parameters of original model


## Summary

| Metric | Original | Pruned (40%) | Improvement |
|--------|----------|--------------|-------------|
| Parameters | 100% | ~18% | **5.5x smaller** |
| Compute (MACs) | 100% | ~63% | **1.6x fewer ops** |
| Accuracy | Baseline | ~1% drop | Minimal impact |

### Parameter Reference

| Parameter | Description | Example |
|-----------|-------------|---------|
| `pruning_ratio` | Percentage of filters to remove | `40` |
| `context` | Pruning scope | `'global'` (whole model) or `'local'` (per-layer) |
| `criteria` | Importance measure | `large_final`, `magnitude`, `taylor` |
| `schedule` | How pruning increases over training | `one_cycle`, `cos`, `linear` |

---

## See Also

- [Pruner](../../prune/pruner.html) - Lower-level API for one-shot pruning
- [Sparsifier](../../sparse/sparsifier.html) - For unstructured sparsification
- [Schedules](../../core/schedules.html) - Available pruning schedules
- [Criteria](../../core/criteria.html) - Filter importance measures
- [YOLO Pruning Tutorial](YOLOV8.html) - Pruning detection models